# Clustering at state level

## imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import PowerTransformer
from utils import add_Loss, clean_data_state, clean_data

from pyclustering.utils.metric import distance_metric, type_metric
from pyclustering.cluster.kmeans import kmeans
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer,random_center_initializer

## Import and clean data

In [2]:
#Select the dataset of one season of one year
YEAR = 2019
SEASON = "Rabi" # or "Kharif" 

#Path to the dataset
pathData = f"Data\RawDataUnified\RawData_{YEAR}_{SEASON}"
print(pathData)


df = pd.read_csv(pathData)

Data\RawDataUnified\RawData_2019_Rabi


C:\Users\marti\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#Clean an add loss to data
df=add_Loss(clean_data_state(df))
df

,State,Crop,Area Sown (Ha),Area Insured (Ha),SI Per Ha (Inr/Ha),Sum Insured (Inr),Indemnity Level,2002 Yield,2003 Yield,2004 Yield,...,2009 Yield,2010 Yield,2011 Yield,2012 Yield,2013 Yield,2014 Yield,2015 Yield,2016 Yield,2017 Yield,Loss
key,,,,,,,,,,,,,,,,,,,,,
andhra pradesh_anantapur_vidapanakal___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,1154.000000,760.00000,561.000000,816.000000,766.000000,670.000000,67.000000,202.000000,746.000000,1.078638e+05
andhra pradesh_anantapur_vajrakarur___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,1154.000000,944.00000,703.000000,842.000000,775.000000,647.000000,450.000000,627.000000,375.000000,0.000000e+00
andhra pradesh_anantapur_gooty___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,1154.000000,752.00000,504.000000,1320.000000,1223.000000,688.000000,814.000000,379.000000,661.000000,1.516659e+06
andhra pradesh_anantapur_guntakal___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,1154.000000,752.00000,504.000000,1320.000000,1223.000000,688.000000,814.000000,379.000000,661.000000,1.516659e+06
andhra pradesh_anantapur_pamidi___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,1154.000000,752.00000,504.000000,1320.000000,1223.000000,688.000000,814.000000,379.000000,661.000000,1.516659e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
west bengal_howrah_uluberia-ii___,12,Wheat,204.149503,0.200000,56810.0,1.136200e+04,0.9,2275.972276,4629.899829,4377.946588,...,2084.770000,1987.31000,1417.770000,2804.980000,2382.030000,2518.480000,1362.950000,2469.040109,2766.065757,0.000000e+00
west bengal_purba medinipur_haldia___,12,Wheat,204.149503,0.750000,38557.0,2.891775e+04,0.9,2275.972276,4629.899829,4377.946588,...,2450.120000,2569.92000,2431.370000,2564.580000,2623.860000,2756.520000,2496.650000,2469.040109,2766.065757,0.000000e+00
west bengal_purba medinipur_mohisadal-ii___,12,Wheat,204.149503,0.750000,38557.0,2.891775e+04,0.9,2275.972276,4629.899829,4377.946588,...,2450.120000,2569.92000,2431.370000,2564.580000,2623.860000,2756.520000,2496.650000,2469.040109,2766.065757,0.000000e+00


## Preprocessing et fonctions utiles

In [19]:
data = pd.get_dummies( df, columns=["Crop"])

for c in data.columns:
    if c != "State":
        pt = PowerTransformer()
        data.loc[:, c] = pt.fit_transform(np.array(data[c]).reshape(-1, 1))

data_list =data.to_numpy()

In [33]:
def tab_cluster(clusters,donnees):
    tab_clusters = np.zeros(len(donnees)) #tab_clusters[i] renvoit le numéro de cluster de la i-ème ligne
    for i in range(k):
        for x in clusters[i]:
            tab_clusters[x] = i
    return tab_clusters

list_state = pd.unique(pd.read_csv(pathData)["State"])

def get_liste_state_cluster(clusters,donnees):
    df2 = donnees.copy(deep = True)
    df2["Cluster"] = tab_cluster(clusters,donnees)
    liste_state_cluster = []
    for i in range(13):
        l = []
        l.append(list_state[i])
        l.append(np.bincount(df2[df2["State"] == i]["Cluster"].to_numpy().astype(int)).argmax())
        liste_state_cluster.append(l)
    return liste_state_cluster

C:\Users\marti\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## K-Means avec distance perso

In [28]:
# Définition d'une distance entre deux parcelles avec forte pénalisation si état différent

def distance_state_penalized(point1, point2):
    if point1[0] == point2[0]:
        result = 0.0
    else:
        result = 9*10**15
    p1_numpy = np.array(point1[1:])
    p2_numpy = np.array(point2[1:])
    result += np.linalg.norm(p1_numpy-p2_numpy)
    return result

metric_perso = distance_metric(type_metric.USER_DEFINED, func=distance_state_penalized)

In [31]:
# Nombre de clusters
k = 6

# Prepare initial centers using K-Means++ method.
initial_centers = kmeans_plusplus_initializer(data_list, k).initialize()
 
# Create instance of K-Means algorithm with prepared centers.
kmeans_instance = kmeans(data_list, initial_centers,metric = metric_perso)
 
# Run cluster analysis and obtain results.
kmeans_instance.process()
clusters = kmeans_instance.get_clusters()
final_centers = kmeans_instance.get_centers()

In [34]:
liste_state_cluster = get_liste_state_cluster(clusters,df)
liste_state_cluster

[['Andhra Pradesh', 2],
 ['Chhattisgarh', 0],
 ['Gujarat', 0],
 ['Haryana', 0],
 ['Karnataka', 1],
 ['Madhya Pradesh', 2],
 ['Maharashtra', 0],
 ['Odisha', 2],
 ['Rajasthan', 0],
 ['Tamil Nadu', 2],
 ['Telangana', 2],
 ['Uttar Pradesh', 0],
 ['West Bengal', 2]]

## K-Means avec définition d'une distance entre états

In [64]:
# regroupement des données par états
liste_df = []
for col in data.columns:
    if col != "State":
        df_by_state_prov = data.groupby("State")
        df_by_state_prov = df_by_state_prov[col].apply(np.array).to_frame()
        liste_df.append(df_by_state_prov[col])

df_by_state = pd.concat(liste_df,axis=1)
df_by_state


,Area Sown (Ha),Area Insured (Ha),SI Per Ha (Inr/Ha),Sum Insured (Inr),Indemnity Level,2002 Yield,2003 Yield,2004 Yield,2005 Yield,2006 Yield,...,Crop_Sunflower IRR,Crop_Sunflower Un-IRR,Crop_Taramira,Crop_Til (Sesame),Crop_Tomato,Crop_Urad,Crop_Urad Un-IRR,Crop_Wheat,Crop_Wheat IRR,Crop_Wheat Un-IRR
State,,,,,,,,,,,,,,,,,,,,,
0,"[0.20916218398224784, 0.20916218398224784, 0.2...","[1.0037981936405962, 1.0037981936405962, 1.003...","[-0.8085527423744752, -0.8085527423744752, -0....","[0.1392766536099348, 0.1392766536099348, 0.139...","[-0.3198520349067133, -0.3198520349067133, -0....","[0.04469661544710228, 0.04469661544710228, 0.0...","[0.5127516709866342, 0.5127516709866342, 0.512...","[0.5446061435624959, 0.5446061435624959, 0.544...","[0.6335774660125708, 0.6335774660125708, 0.633...","[-1.9721020411963335, -1.9721020411963335, -1....",...,"[-0.03317245525030668, -0.03317245525030668, -...","[-0.03383201008439359, -0.03383201008439359, -...","[-0.03891125452610982, -0.03891125452610982, -...","[-0.027099635919418653, -0.027099635919418653,...","[-0.009391268334929882, -0.009391268334929882,...","[-0.03903284747662696, -0.03903284747662696, -...","[-0.006398919559813339, -0.006398919559813339,...","[-0.4960667663993583, -0.4960667663993583, -0....","[-0.25729600938596, -0.25729600938596, -0.2572...","[-0.10462685378575722, -0.10462685378575722, -..."
1,"[0.20916218398224784, 0.20916218398224784, 0.2...","[1.749281624952688, 1.749281624952688, 1.74928...","[-2.6891035672027668, -2.6891035672027668, -2....","[0.10019879215620414, 0.10019879215620414, 0.1...","[-0.3198520349067133, -0.3198520349067133, -0....","[0.04469661544710228, 0.04469661544710228, 0.0...","[0.5127516709866342, 0.5127516709866342, 0.512...","[0.5446061435624959, 0.5446061435624959, 0.544...","[0.6335774660125708, 0.6335774660125708, 0.633...","[0.6465804074530026, 0.6465804074530026, 0.646...",...,"[-0.03317245525030668, -0.03317245525030668, -...","[-0.03383201008439359, -0.03383201008439359, -...","[-0.03891125452610982, -0.03891125452610982, -...","[-0.027099635919418653, -0.027099635919418653,...","[-0.009391268334929882, -0.009391268334929882,...","[-0.03903284747662696, -0.03903284747662696, -...","[-0.006398919559813339, -0.006398919559813339,...","[-0.4960667663993583, -0.4960667663993583, -0....","[-0.25729600938596, -0.25729600938596, -0.2572...","[-0.10462685378575722, -0.10462685378575722, -..."
2,"[7.050761498846274, 7.050761498846274, 7.05076...","[1.915209111952602, 1.915209111952602, 1.91520...","[-0.8085527423744752, -0.8085527423744752, -0....","[3.6354085167257915, 3.6354085167257915, 3.635...","[-0.3198520349067133, -0.3198520349067133, -0....","[0.04469661544710228, 0.04469661544710228, 0.0...","[0.5127516709866342, 0.5127516709866342, 0.512...","[0.5446061435624959, 0.5446061435624959, 0.544...","[0.6335774660125708, 0.6335774660125708, 0.633...","[-0.27025706315655096, -0.23473732115032503, -...",...,"[-0.03317245525030668, -0.03317245525030668, -...","[-0.03383201008439359, -0.03383201008439359, -...","[-0.03891125452610982, -0.03891125452610982, -...","[-0.027099635919418653, -0.027099635919418653,...","[-0.009391268334929882, -0.009391268334929882,...","[-0.03903284747662696, -0.03903284747662696, -...","[-0.006398919559813339, -0.006398919559813339,...","[-0.4960667663993583, -0.4960667663993583, -0....","[-0.25729600938596, -0.25729600938596, -0.2572...","[-0.10462685378575722, -0.10462685378575722, -..."
3,"[0.20916218398224784, 0.20916218398224784, 0.2...","[-0.6705790084849246, -0.6705790084849246, -0....","[-0.1877826320820115, -0.1877826320820115, -0....","[-2.47638670683907, -2.47638670683907, -2.4763...","[1.5912586968619373, 1.5912586968619373, 1.591...","[0.04469661544710228, 0.04469661544710228, 0.0...","[0.5127516709866342, 0.5127516709866342, 0.512...","[0.5446061435624959, 0.5446061435624959, 0.544...","[0.6335774660125708, 0.6335774660125708, 0.633...","[0.6465804074530026, 0.6465804074530026, 0.646...",...,"[-0.03

In [68]:
# Nombre de clusters
k = 3

# Prepare initial centers using K-Means++ method.
initial_centers = random_center_initializer(df_by_state, k).initialize()
 
# Create instance of K-Means algorithm with prepared centers.
kmeans_instance = kmeans(df_by_state, initial_centers)
 
# Run cluster analysis and obtain results.
kmeans_instance.process()
clusters = kmeans_instance.get_clusters()
final_centers = kmeans_instance.get_centers()

KeyError: 8

In [ ]:
# Distance entre états



## DBSCAN

In [47]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN(metric=metric_perso).fit(pd.get_dummies( df, columns=["Crop"]).to_numpy())

KeyboardInterrupt: 

In [46]:
len(np.unique(clustering.labels_))

8001